# Doğrusal Bağlanım (Linear Regression)
yazan Marc Deisenroth, çeviren Utku Karaca

Doğrusal bağlanım problemi:
$$
y = \boldsymbol x^T\boldsymbol\theta + \epsilon\,,\quad \epsilon \sim \mathcal N(0, \sigma^2)
$$

$\boldsymbol x\in\mathbb{R}^D$ girdiler ve $y\in\mathbb{R}$ gürültülü gözlemler. Parametre vektörü $\boldsymbol\theta\in\mathbb{R}^D$ ise fonksiyonumuzu belirleyen vektör.

Eğitim kümesi $(\boldsymbol x_n, y_n)$, $n=1,\ldots, N$, sırasıyla, kısa gösterim olarak $\mathcal X = \{\boldsymbol x_1, \ldots, \boldsymbol x_N\}$ ve eşlenik eğitim hedefleri $\mathcal Y = \{y_1, \ldots, y_N\}$.

Bu eğitimde amacımız iyi $\boldsymbol\theta$ parametrelerine ulaşmak.

Gerekli paketler:

In [ ]:
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt
%matplotlib inline

Eğitim kümesi

In [ ]:
# Eğitim kümesinin tanımlanması
X = np.array([-3, -1, 0, 1, 3]).reshape(-1,1) # 5x1 vektör, N=5, D=1
y = np.array([-1.2, -0.7, 0.14, 0.67, 1.67]).reshape(-1,1) # 5x1 vektör

# Eğitim kümesinin grafik üzerinde gösterilmesi
plt.figure()
plt.plot(X, y, '+', markersize=10)
plt.xlabel("$x$")
plt.ylabel("$y$");

## 1. En Büyük Olabilirlik (Maximum Likelihood) 
$\boldsymbol\theta$ parametresinin en büyük olabilirlik kestirimi ile başlayacağız. Olabilirliği en çoklayan $\boldsymbol\theta^{\mathrm{ML}}$ parametrelerini bulacağız. 

Olabilirlik fonksiyonu:
$$
p(\mathcal Y | \mathcal X, \boldsymbol\theta) = \prod_{n=1}^N p(y_n | \boldsymbol x_n, \boldsymbol\theta)\,.
$$
Olabilirlik fonksiyonunu en çoklayan analitik ifade:
$$
\boldsymbol\theta^{\text{ML}} = (\boldsymbol X^T\boldsymbol X)^{-1}\boldsymbol X^T\boldsymbol y\in\mathbb{R}^D\,,
$$
 
$$
\boldsymbol X = [\boldsymbol x_1, \ldots, \boldsymbol x_N]^T\in\mathbb{R}^{N\times D}\,,\quad \boldsymbol y = [y_1, \ldots, y_N]^T \in\mathbb{R}^N\,.
$$

In [ ]:
def max_lik_kestirimi(X, y):
    # X: N x D eğitim kümesi matrisi
    # y: N x 1 eğitim hedefler/gözlemler vektörü
    # dön: en büyük olabilirlik parametreleri maximum likelihood parameters (D x 1)
    
    theta_ml = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    return theta_ml

In [ ]:
# en büyük olabilirlik kestirimi
theta_ml = max_lik_kestirimi(X, y)
print(theta_ml)

En büyük olabilirlik kestirimini kullanarak tahmin yapma:

In [ ]:
def kestirim_ile_tahmin(Xtest, theta):
    
    # Xtest: K x D test kümesi matrisi
    # theta: D x 1 parametre vektörü
    # dön: f(Xtest) tahmini; K x 1 vektör 
    
    tahmin = Xtest.dot(theta)
    
    return tahmin

Sonuçlar:

In [ ]:
# test kümesi tanımlanması
#     100 x 1 vektör
Xtest = np.linspace(-5,5,100).reshape(-1,1)

# En büyük olabilirlik kestirici (Maximum Likelihood Estimator)
# ile test noktalarındaki fonksiyon değerini tahmin etme
ml_tahmini = kestirim_ile_tahmin(Xtest, theta_ml)

# grafik
plt.figure()
plt.plot(X, y, '+', markersize=10)
plt.plot(Xtest, ml_tahmini)
plt.xlabel("$x$")
plt.ylabel("$y$");

Farklı bir eğitim kümesine bakalım.

In [ ]:
y_yeni = y + 2.0

plt.figure()
plt.plot(X, y_yeni, '+', markersize=10)
plt.xlabel("$x$")
plt.ylabel("$y$");

In [ ]:
# en büyük olabilirlik kestirimleri
theta_ml = max_lik_kestirimi(X, y_yeni)
print(theta_ml)

# test kümesinin tanımlanması
Xtest = np.linspace(-5,5,100).reshape(-1,1)

# En büyük olabilirlik kestirici (Maximum Likelihood Estimator)
# ile test noktalarındaki fonksiyon değerini tahmin etme
ml_tahmini = kestirim_ile_tahmin(Xtest, theta_ml)

# plot
plt.figure()
plt.plot(X, y_yeni, '+', markersize=10)
plt.plot(Xtest, ml_tahmini)
plt.xlabel("$x$")
plt.ylabel("$y$");

#### Soru:
1. Yapılan son kestirim iyi değil: Turuncu çizgi gözlem noktalarından çok uzak. Neden?
2. Bu problemi nasıl giderebiliriz?

Yeni bir doğrusal bağlanım modeli: Daha esnek model. 
$$
y = \theta_0 + \boldsymbol x^T \boldsymbol\theta_1 + \epsilon\,,\quad \epsilon\sim\mathcal N(0,\sigma^2)
$$
İlk modele bir yanlılık parametresi ekledik.
#### Question:
1. Bu yanlılık parametresinin etkisi nedir? Nasıl bir esneklik sağlayacaktır?

Yanlılık parametresini hesaba katarak genişletilmiş girdi vektörümüz:
$\boldsymbol x_{\text{aug}} = \begin{bmatrix}1\\\boldsymbol x\end{bmatrix}$. Yeni modelimiz:  
$$
y = \boldsymbol x_{\text{aug}}^T\boldsymbol\theta_{\text{aug}} + \epsilon\,,\quad \boldsymbol\theta_{\text{aug}} = \begin{bmatrix}
\theta_0\\
\boldsymbol\theta_1
\end{bmatrix}\,.
$$

In [ ]:
N, D = X.shape
# genişletilmiş eğitim kümesi matrisi: N x (D+1)
X_gen = np.hstack([np.ones((N,1)), X])
# yeni theta vektörü: (D+1) x 1
theta_gen = np.zeros((D+1, 1)) 

Yeni modelimiz için en büyük olabilirlik kestirici:

In [ ]:
def max_lik_kestirici_gen(X_gen, y):
    
    theta_gen_ml = max_lik_kestirimi(X_gen, y)
    
    return theta_gen_ml

In [ ]:
theta_gen_ml = max_lik_kestirici_gen(X_gen, y_yeni)
print(theta_gen_ml)

Yeni tahminler:

In [ ]:
# test kümesi yaratılması 
# (girdileri 1'ler ile genişletmemiz gerekiyor)
Xtest_gen = np.hstack([np.ones((Xtest.shape[0],1)), Xtest])

# En büyük olabilirlik kestirici (Maximum Likelihood Estimator)
# ile test noktalarındaki fonksiyon değerini tahmin etme
ml_tahmin = kestirim_ile_tahmin(Xtest_gen, theta_gen_ml)

# plot
plt.figure()
plt.plot(X, y_yeni, '+', markersize=10)
plt.plot(Xtest, ml_tahmin)
plt.xlabel("$x$")
plt.ylabel("$y$");

## Doğrusal olmayan öznitelikler (Nonlinear Features)
Doğrusal bağlanım doğrusal olmayan fonksiyoları tahmin etmede de kullanılabilir. Önemli olan $\boldsymbol\theta$ parametresinin doğrusal olarak uygulanmasıdır. $\boldsymbol x$ doğrusal olmayan girdiler içerebilir.

Doğrusal bağlanım ile öğrenebileceğimiz fonksiyon gösterimi:
$$
f(\boldsymbol x, \boldsymbol\theta) = \sum_{k = 1}^K \theta_k \phi_k(\boldsymbol x)\,,
$$
Öznitelikler ($\phi_k(\boldsymbol x)$) $\boldsymbol x$'lerin (muhtemelen doğrusal olmayan) dönüşümünü göstermektedir.

Örnek:

In [ ]:
y = np.array([10.05, 1.5, -1.234, 0.02, 8.03]).reshape(-1,1)
plt.figure()
plt.plot(X, y, '+')
plt.xlabel("$x$")
plt.ylabel("$y$");

#### Polinomsal Bağlanım (Polynomial Regression)
Polinomları doğrusal bağlanım kullanarak tahminleyebiliriz çünkü bir polinomu $K$ dereceli yazabiliriz:

$$\sum_{k=0}^K \theta_k x^k = \boldsymbol \phi(x)^T\boldsymbol\theta\,,\quad
\boldsymbol\phi(x)= 
\begin{bmatrix}
x^0\\
x^1\\
\vdots\\
x^K
\end{bmatrix}\in\mathbb{R}^{K+1}\,.
$$
$\boldsymbol\phi(x)$:$x\in\mathbb{R}$'lerin doğrusal olmayan öznitelik dönüşümü.

Bütün öznitelik dönüşümlerini bir matris altında toplayabiliriz:
$$
\boldsymbol\Phi = \begin{bmatrix}
\boldsymbol\phi(x_1) & \boldsymbol\phi(x_2) & \cdots & \boldsymbol\phi(x_n)
\end{bmatrix}^T \in\mathbb{R}^{N\times K+1}
$$

$\boldsymbol\Phi$ öznitelik matrisi:

In [ ]:
def poli_oznitelikleri(X, K):
    
    # X: girdiler N x 1
    # K: polinom derecesi 
    # öznitelik matrisi Phi'yi hesaplar (N x (K+1))
    
    X = X.flatten()
    N = X.shape[0]
    
    # Phi matrisinin yaratılması
    Phi = np.zeros((N, K+1))
    
    # Öznitelik matrisinin hesaplanması
    for k in range(K+1):
        Phi[:,k] = X**k #X^k
    return Phi

Öznitelik matrisini kullanarak en büyük olabilirlik kestiricisini aşağıdaki gibi hesaplayabiliriz:
$$
\boldsymbol \theta^\text{ML} = (\boldsymbol\Phi^T\boldsymbol\Phi)^{-1}\boldsymbol\Phi^T\boldsymbol y
$$

Genellikle sayısal kararlılık için $\boldsymbol\Phi^T\boldsymbol\Phi$ terimine küçük diyagonal seğirme ($\kappa>0$) ekliyoruz. Bu sayede önemli bir sıkıntı yaşamadan tersini alabiliyoruz. Böylece en büyük olabilirlik kestirimi:
$$
\boldsymbol \theta^\text{ML} = (\boldsymbol\Phi^T\boldsymbol\Phi + \kappa\boldsymbol I)^{-1}\boldsymbol\Phi^T\boldsymbol y
$$

In [ ]:
def dog_olm_oznitelik_ml(Phi, y):
    # Phi: eğitim kümesi için öznitelik matrisi, N x D
    # y: eğitim kümesi hedefleri, N x 1
    # dön: en büyük olabilirlik kestiricisi theta_ml, D x 1
    
    kappa = 1e-08 # seğirme terimi; sayısal kararlılık için
    
    D = Phi.shape[1]  
    
    # en büyük olabilirlik kestiricisi
    Pt = Phi.T.dot(y) # Phi^T*y
    PP = Phi.T.dot(Phi) + kappa*np.eye(D) # Phi^T*Phi + kappa*I
        
    # en büyük olabilirlik kestiricisi
    C = scipy.linalg.cho_factor(PP)
    theta_ml = scipy.linalg.cho_solve(C, Pt) # inv(Phi^T*Phi + kappa*I)*Phi^T*y 
    
    return theta_ml

Eldekiler:
* Öznitelik matrisi
* Polinomsal bağlanım için en büyük olabilirlik kestiricisi

Test seti $\boldsymbol X_{\text{test}}\in\mathbb{R}$ tahmini yapabilmek için $\boldsymbol X_{\text{test}}$'in öznitelik dönüşümlerini yapmamız gerekiyor. 

$$\boldsymbol\Phi_{\text{test}}= \boldsymbol\phi(\boldsymbol X_{\text{test}})$$

In [ ]:
K = 2 # uyumlamak istediğimiz polinom derecesi
Phi = poli_oznitelikleri(X, K) # öznitelik matrisi, N x (K+1)

theta_ml = dog_olm_oznitelik_ml(Phi, y) # en büyük olabilirlik kestiricisi

# test veri kümesinin yaratılması
Xtest = np.linspace(-4,4,100).reshape(-1,1)

# test girdileri için öznitelik matrisi
Phi_test = poli_oznitelikleri(Xtest, K)

y_tahmin = Phi_test.dot(theta_ml) # tahmin edilen y değerleri

plt.figure()
plt.plot(X, y, '+')
plt.plot(Xtest, y_tahmin)
plt.xlabel("$x$")
plt.ylabel("$y$");

print(theta_ml)

## Model Kalitesini Değerlendirme
Yeni bir veri kümesi:

In [ ]:
def f(x):   
    return np.cos(x) + 0.2*np.random.normal(size=(x.shape))

X = np.linspace(-4,4,20).reshape(-1,1)
y = f(X)

plt.figure()
plt.plot(X, y, '+')
plt.xlabel("$x$")
plt.ylabel("$y$");

Polinomsal bağlanım kodunu kullanarak bu veri setini tahmin edelim.

In [ ]:
K = 1 # uyumlamak istediğimiz polinom derecesi

Phi = poli_oznitelikleri(X, K) # öznitelik matrisi, N x (K+1)

theta_ml = dog_olm_oznitelik_ml(Phi, y) # en büyük olabilirlik kestiricisi

# test kümesi yaratılması
Xtest = np.linspace(-5,5,100).reshape(-1,1)
ytest = f(Xtest) # gerçek y değerleri

# test girdileri için öznitelik matrisi
Phi_test = poli_oznitelikleri(Xtest, K)

y_tahmin = Phi_test.dot(theta_ml) # tahmini y değerleri

# plot
plt.figure()
plt.plot(X, y, '+')
plt.plot(Xtest, y_tahmin)
plt.plot(Xtest, ytest)
plt.legend(["gözlem", "kestirim", "gerçek gözlem (Test)"])
plt.xlabel("$x$")
plt.ylabel("$y$");

Yöntemli bir şekilde model kalitesini değerlendirebilir miyiz? 

Ortalama karesel hata (OKH): 
$$
\text{OKH} = \sqrt{\frac{1}{N}\sum_{n=1}^N(y_n - y_n^\text{pred})^2}
$$

Fonksiyonu:

In [ ]:
def OKH(y, y_tahmin):
    okh = np.sqrt(np.mean((y-y_tahmin)**2))
    return okh

Farklı polinom dereceleri için OKH hesaplanması:

In [ ]:
K_max = 20
okh_egitim = np.zeros((K_max+1,))

for k in range(K_max+1):
    # öznitelik matrisi
    Phi = poli_oznitelikleri(X, k)
    
    # en büyük olabilirlik kestirici (maximum likelihood estimate)
    theta_ml = dog_olm_oznitelik_ml(Phi, y)
    
    # eğitim kümesi y değerlerinin tahmini
    y_tahmin_egitim = Phi.dot(theta_ml)
    
    # RMSE on training set
    okh_egitim[k] = OKH(y, y_tahmin_egitim)
    

plt.figure()
plt.plot(okh_egitim)
plt.xlabel("polinom derecesi")
plt.ylabel("OKH");

Eğitim kümesinde en iyi sonucu veren polinom derecesinin performansını test kümesinde inceleyelim.

In [ ]:
plt.figure()
plt.plot(X, y, '+')

# öznitelik matrisi
Phi = poli_oznitelikleri(X, 5)

# en büyük olabilirlik kestirici (maximum likelihood estimate)
theta_ml = dog_olm_oznitelik_ml(Phi, y)   

# test kümesi için y değerlerinin tahmini
Phi_test = poli_oznitelikleri(Xtest, 5)

y_tahmin_test = Phi_test.dot(theta_ml)

plt.plot(Xtest, y_tahmin_test) 
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.legend(["veri", "en iyi en büyük olabilirlik (maximum likelihood fit)"]);

Modelin gerçek performansını görmek için eğitim kümesi üzerindeki OKH yanıltıcı olabilir. Bu sebeple test kümesi üzerinde en düşük OKH değerini veren dereceyi seçmeliyiz.

In [ ]:
K_max = 20
okh_egitim = np.zeros((K_max+1,))
okh_test = np.zeros((K_max+1,))

for k in range(K_max+1):
    
    # öznitelik matrisi
    Phi = poli_oznitelikleri(X, k)
    
    # en büyük olabilirlik kestirici (maximum likelihood estimate)
    theta_ml = dog_olm_oznitelik_ml(Phi, y)
    
    # eğitim kümesi y değer tahminleri
    y_tahmin_egitim = Phi.dot(theta_ml)
    
    # eğitim kümesi OKH değeri
    okh_egitim[k] = OKH(y, y_tahmin_egitim)    
    
    # test kümesi için öznitelik matrisi
    Phi_test = poli_oznitelikleri(Xtest, k)
    
    # test kümesi tahmini
    y_tahmin_test = Phi_test.dot(theta_ml)
    
    # test kümesi OKH değeri on test set
    okh_test[k] = OKH(ytest, y_tahmin_test)
    

plt.figure()
plt.semilogy(okh_egitim) # logaritmik ölçekte OKH 
plt.semilogy(okh_test) # logaritmik ölçekte OKH 
plt.xlabel("polinom derecesi")
plt.ylabel("OKH")
plt.legend(["eğitim kümesi", "test kümesi"]);

En iyi polinom derecesi:

In [ ]:
plt.figure()
plt.plot(X, y, '+')
k = 5
# öznitelik matrisi
Phi = poli_oznitelikleri(X, k)

# en büyük olabilirlik kestirici (maximum likelihood estimate)
theta_ml = dog_olm_oznitelik_ml(Phi, y)   

# test kümesi için öznitelik matrisi
Phi_test = poli_oznitelikleri(Xtest, k)

y_tahmin_test = Phi_test.dot(theta_ml)

plt.plot(Xtest, y_tahmin_test) 
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.legend(["veri", "en büyük olabilirlik kestirici (maximum likelihood estimate)"]);

## 2. En Büyük Posteriyori Kestirim (Maximum A Posteriori Estimation, MAP Estimation)
Model:
$$
y = \boldsymbol\phi(\boldsymbol x)^T\boldsymbol\theta + \epsilon\,,\quad \epsilon\sim\mathcal N(0,\sigma^2)\,.
$$
Gürültü varyansının ($\sigma^2$) bildiğimizi varsayıyoruz.

Olabilirliği en büyüklemek yerine parametrelerin ($\boldsymbol\theta$) sonsal posteriyori dağılımını en büyükleyebiliriz:
$$
p(\boldsymbol\theta|\mathcal X, \mathcal Y) = \frac{\overbrace{p(\mathcal Y|\mathcal X, \boldsymbol\theta)}^{\text{olabilirlik (likelihood)}}\overbrace{p(\boldsymbol\theta)}^{\text{önsel (prior)}}}{\underbrace{p(\mathcal Y|\mathcal X)}_{\text{kanıt(evidence)}}}
$$
Önsel olasılık parametresinin ($p(\boldsymbol\theta)$) amacı parametrelerin ekstrem değerler almasını engellemektir, örneğin aşırı uyumlama (overfitting). Önsellik "makul" bir parametre değerler aralığı belirlememizi sağlar. Genellikle Gauss (ortalaması $\boldsymbol 0$ ve varyansı $\alpha^2$) önselini tercih ediyoruz $\mathcal N(\boldsymbol 0, \alpha^2\boldsymbol I)$.

Parametrelerin MAP kestirimi:
$$
\boldsymbol\theta^{\text{MAP}} = (\boldsymbol\Phi^T\boldsymbol\Phi + \frac{\sigma^2}{\alpha^2}\boldsymbol I)^{-1}\boldsymbol\Phi^T\boldsymbol y
$$
$\sigma^2$: gürültü varyansı.

In [ ]:
def map_kestirim_poli(Phi, y, sigma, alpha):
    # Phi: eğitim kümesi, N x D
    # y: eğitim hedef değerleri, D x 1
    # sigma: gürültünün standart sapma 
    # alpha: parametre önselliğinin standart sapması
    # dön: MAP kestirimi theta_map, Size of D x 1
    
    D = Phi.shape[1] 
    
    PP = Phi.T.dot(Phi) + (sigma/alpha)**2 * np.eye(D)
    theta_map = scipy.linalg.solve(PP, Phi.T.dot(y))
    
    return theta_map

In [ ]:
# tahmin etmeye çalıştığımız fonksiyon
def g(x, sigma):
    p = np.hstack([x**0, x**1, np.sin(x)])
    w = np.array([-1.0, 0.1, 1.0]).reshape(-1,1)
    return p.dot(w) + sigma*np.random.normal(size=x.shape) 

In [ ]:
# veri yaratımı
sigma = 1.0 # gürültü standart sapması
alpha = 1.0 # parametre önselliğinin standart sapması
N = 20

np.random.seed(42)

X = (np.random.rand(N)*10.0 - 5.0).reshape(-1,1)
y = g(X, sigma) # eğitim kümesi hedef değerleri

plt.figure()
plt.plot(X, y, '+')
plt.xlabel("$x$")
plt.ylabel("$y$");

In [ ]:
# MAP estimate kestirimleri
K = 8 # polinom derecesi   

# öznitelik matrisi
Phi = poli_oznitelikleri(X, K)

theta_map = map_kestirim_poli(Phi, y, sigma, alpha)

# en büyük olabilirlik kestirimi (maximum likelihood estimate)
theta_ml = dog_olm_oznitelik_ml(Phi, y)

Xtest = np.linspace(-5,5,100).reshape(-1,1)
ytest = g(Xtest, sigma)

Phi_test = poli_oznitelikleri(Xtest, K)
y_tahmin_map = Phi_test.dot(theta_map)

y_tahmin_mle = Phi_test.dot(theta_ml)

plt.figure()
plt.plot(X, y, '+')
plt.plot(Xtest, y_tahmin_map)
plt.plot(Xtest, g(Xtest, 0))
plt.plot(Xtest, y_tahmin_mle)

plt.legend(["veri", "map tahmini", "gerçek fonksiyon", "en büyük olabilirlik"]);

In [ ]:
print(np.hstack([theta_ml, theta_map]))

Farklı polinom dereceleri için OKH hesaplanması: MAP kestirimi en büyük olabilirlikte karşılaştığımız aşırı uyumlama için bir çözüm mü?

In [ ]:
K_max = 12 # deneyeceğimiz en büyük polinom derecesi

okh_mle = np.zeros((K_max+1,))
okh_map = np.zeros((K_max+1,))

for k in range(K_max+1):
    
    # öznitelik matrisi
    Phi = poli_oznitelikleri(X, k)
    
    # en büyük olabilirlik kestirimi (maximum likelihood estimate)
    theta_ml = dog_olm_oznitelik_ml(Phi, y)
    
    # test kümesi için öznitelik matrisi
    Phi_test = poli_oznitelikleri(Xtest, k)
    
    # test noktalarında fonsiyon değerini tahmin etme
    # (en büyük olabilirlik - maximum likelihood)
    y_tahmin_test_mle = Phi_test.dot(theta_ml)
    
    # test kümesi OKH değeri (en büyük olabilirlik - maximum likelihood)
    okh_mle[k] = OKH(ytest, y_tahmin_test_mle)
    
    # MAP kestirimi
    theta_map = map_kestirim_poli(Phi, y, sigma, alpha)

    # öznitelik matrisi
    Phi_test = poli_oznitelikleri(Xtest, k)
    
    # test noktalarında fonsiyon değerini tahmin etme (MAP)
    y_tahmin_test_map = Phi_test.dot(theta_map)
    
    # test kümesi OKH değeri (MAP)
    okh_map[k] = OKH(ytest, y_tahmin_test_map)
    
plt.figure()
plt.semilogy(okh_mle) # logaritmik ölçekte OKH 
plt.semilogy(okh_map) # logaritmik ölçekte OKH 
plt.xlabel("polinom derecesi")
plt.ylabel("OKH")
plt.legend(["En Büyük Olabilirlik (Maximum likelihood)", "MAP"])

## 3. Bayesyen Doğrusal Bağlanım (Bayesian Linear Regression)

In [ ]:
# Test kümesi
Ntest = 200
Xtest = np.linspace(-5, 5, Ntest).reshape(-1,1) # test inputs

onsel_var = 2.0 # parametre önselliğinin varyansı (alpha^2). Bilindiği varsayılıyor.
gurultu_var = 1.0 # gürültü varyansı (sigma^2). Bilindiği varsayılıyor.

pol_der = 3 # polinom derecesi

Parametre önselliği varsayımı: $p(\boldsymbol\theta) = \mathcal N (\boldsymbol 0, \alpha^2\boldsymbol I)$. Her bir test girdisi $\boldsymbol x_*$ için, elde ettiğimiz önsel ortalama: 
$$
E[f(\boldsymbol x_*)] = 0
$$
ve önsel (marjinal) varyans (gürültüyü yoksayarak)
$$
V[f(\boldsymbol x_*)] = \alpha^2\boldsymbol\phi(\boldsymbol x_*) \boldsymbol\phi(\boldsymbol x_*)^\top
$$
$\boldsymbol\phi(\cdot)$: öznitelik eşlemi (feature map).

In [ ]:
# test kümesi öznitelik matrisinin hesaplanması
Phi_test = poli_oznitelikleri(Xtest, pol_der) # özniteli matrisi, N x (pol_der+1)

# test girdiler (marjinal) önselliğinin hesaplanması
# önsel ortalaması
onsel_ortalama = np.zeros((Ntest,1))

# önsel varyansı
ful_kovaryans = Phi_test.dot(Phi_test.T) * onsel_var # bütün fonksiyon değerler kovaryans matrisi, N x N
onsel_marjinal_var =  np.diag(ful_kovaryans)

# Let us visualize the prior over functions
plt.figure()
plt.plot(Xtest, onsel_ortalama, color="k")

guven_sinir1 = np.sqrt(onsel_marjinal_var).flatten()
guven_sinir2 = 2.0*np.sqrt(onsel_marjinal_var).flatten()
guven_sinir3 = 2.0*np.sqrt(onsel_marjinal_var + gurultu_var).flatten()
plt.fill_between(Xtest.flatten(), onsel_ortalama.flatten() + guven_sinir1, 
             onsel_ortalama.flatten() - guven_sinir1, alpha = 0.1, color="k")
plt.fill_between(Xtest.flatten(), onsel_ortalama.flatten() + guven_sinir2, 
                 onsel_ortalama.flatten() - guven_sinir2, alpha = 0.1, color="k")
plt.fill_between(Xtest.flatten(), onsel_ortalama.flatten() + guven_sinir3, 
                 onsel_ortalama.flatten() - guven_sinir3, alpha = 0.1, color="k")

plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title("Prior over functions");

Now, we will use this prior distribution and sample functions from it.

In [ ]:
# önselden örnekler
ornek_sayisi = 10

# parametre önselinden örneklenen rasgele ağırlıklar yaratmamız gerekiyor
rasgele_agirlik = np.random.normal(size=(pol_der+1,ornek_sayisi), scale=np.sqrt(onsel_var))

# Now, we compute the induced random functions, evaluated at the test input locations
# Every function sample is given as f_i = Phi * theta_i, 
# where theta_i is a sample from the parameter prior

ornek_fonksiyon = Phi_test.dot(rasgele_agirlik)

plt.figure()
plt.plot(Xtest, ornek_fonksiyon, color="r")
plt.title("Ölcüle göre olası fonsiyonlar")
print("Örneklenen bütün fonsiyonlar "+str(pol_der)+". dereceden fonsiyonlardır.");

Eğitim gözlemlerini $\boldsymbol x_1, \dotsc, \boldsymbol x_N$ bir matris içerisinde toplama: $\boldsymbol X = [\boldsymbol x_1, \dotsc, \boldsymbol x_N]^\top\in\mathbb{R}^{N\times D}$

In [ ]:
N = 10
X = np.random.uniform(high=5, low=-5, size=(N,1)) # eğitim kümesi, N x 1
y = g(X, np.sqrt(gurultu_var)) # eğitim hedef değerleri, N x 1

Posteriyor hesaplaması:

In [ ]:
def poliUydur(X, y, K, onsel_var, gurultu_var):
    # X: eğitim kümesi, N x D
    # y: eğitim hedef değerleri, N x 1
    # K: polinom derecesi
    # onsel_var: parametre dağılımının önsel varyansı
    # sigma: gürültü varyansı
    
    segirme = 1e-08 # # seğirme terimi; sayısal kararlılık için
    
    Phi = poli_oznitelikleri(X, K) # öznitelik matrisi, N x (K+1) 
    
    # en büyük olabilirlik kestirimi hesaplaması
    Pt = Phi.T.dot(y) # Phi*y, size (K+1,1)
    PP = Phi.T.dot(Phi) + segirme*np.eye(K+1) #(K+1, K+1)
    C = scipy.linalg.cho_factor(PP)
    # en büyük olabilirlik kestirimi (maximum likelihood estimate)
    theta_ml = scipy.linalg.cho_solve(C, Pt) # inv(Phi^T*Phi)*Phi^T*y, size (K+1,1)
    
#     theta_ml = scipy.linalg.solve(PP, Pt) # inv(Phi^T*Phi)*Phi^T*y, size (K+1,1)
    
    # MAP kestirimi
    theta_map = scipy.linalg.solve(PP + gurultu_var/onsel_var*np.eye(K+1), Pt)
    
    # parametre posteriyoru
    iSN = (np.eye(K+1)/onsel_var + PP/gurultu_var) # posteriyor hassaslığı
    SN = scipy.linalg.pinv(gurultu_var*np.eye(K+1)/onsel_var + PP)*gurultu_var  # posteriyor kovaryansı
    mN = scipy.linalg.solve(iSN, Pt/gurultu_var) # posteriyor ortalaması
    
    return (theta_ml, theta_map, mN, SN)

In [ ]:
theta_ml, theta_map, theta_ortalama, theta_var = poliUydur(X, y, pol_der, alpha, sigma)

3 farklı kestiricilerden kestirimler:
\begin{align}
&\text{En Büyün Olabilirlik (Maximum likelihood): }E[f(\boldsymbol X_{\text{test}})] = \boldsymbol \phi(X_{\text{test}})\boldsymbol \theta_{ml}\\
&\text{En Büyük Posteriyori (Maximum a posteriori): } E[f(\boldsymbol X_{\text{test}})] = \boldsymbol \phi(X_{\text{test}})\boldsymbol \theta_{map}\\
&\text{Bayesyen: } p(f(\boldsymbol X_{\text{test}})) = \mathcal N(f(\boldsymbol X_{\text{test}}) \,|\, \boldsymbol \phi(X_{\text{test}}) \boldsymbol\theta_{\text{mean}},\, \boldsymbol\phi(X_{\text{test}}) \boldsymbol\theta_{\text{var}}  \boldsymbol\phi(X_{\text{test}})^\top)
\end{align}

In [ ]:
# kestirimler (ölçüm/gözlem gürültülerini yoksayarak)
Phi_test = poli_oznitelikleri(Xtest, pol_der) # N x (K+1)

# en büyük olabilirlik kestirimleri (sadece ortalama)
m_mle_test = Phi_test.dot(theta_ml)

# MAP kestirimleri (sadece ortalama)
m_map_test = Phi_test.dot(theta_map)

# kestirim dağılımı (Bayesyen Doğrusal Bağlanım, Bayesian linear regression)
# ortalama kestirimi
ortalama_bdb = Phi_test.dot(theta_ortalama)
# varyans kestirimi
kov_bdb =  Phi_test.dot(theta_var).dot(Phi_test.T)

In [ ]:
# posteriyor çizimi
plt.figure()
plt.plot(X, y, "+")
plt.plot(Xtest, m_mle_test)
plt.plot(Xtest, m_map_test)
var_bdb = np.diag(kov_bdb)
guven_sinir1 = np.sqrt(var_bdb).flatten()
guven_sinir2 = 2.0*np.sqrt(var_bdb).flatten()
guven_sinir3 = 2.0*np.sqrt(var_bdb + sigma).flatten()

plt.fill_between(Xtest.flatten(), ortalama_bdb.flatten() + guven_sinir1, 
                 ortalama_bdb.flatten() - guven_sinir1, alpha = 0.1, color="k")
plt.fill_between(Xtest.flatten(), ortalama_bdb.flatten() + guven_sinir2, 
                 ortalama_bdb.flatten() - guven_sinir2, alpha = 0.1, color="k")
plt.fill_between(Xtest.flatten(), ortalama_bdb.flatten() + guven_sinir3, 
                 ortalama_bdb.flatten() - guven_sinir3, alpha = 0.1, color="k")
plt.legend(["Eğitim Kümesi", "MLE (En büyük olabilirlik)", "MAP", "BDB"])
plt.xlabel('$x$');
plt.ylabel('$y$');